In [1]:
import numpy as np

import dask
import dask.array as da
from dask.distributed import Client
import matplotlib.pyplot as plt
import concurrent.futures
import cv2
import time
import tensorflow as tf

import h5py

In [3]:
# client = Client(processes=False)
# client

In [4]:
hd = h5py.File('../../DQN/car_racing.hdf5', 'r')
observations = da.from_array(hd['observations'], chunks=(64, 1000, 16, 16, 3))
observations

,Array,Chunk
Bytes,294.91 GB,196.61 MB
Shape,"(1500, 1000, 128, 128, 3)","(64, 1000, 16, 16, 3)"
Count,1537 Tasks,1536 Chunks
Type,float32,numpy.ndarray


In [5]:
o = hd['observations']

In [169]:
Z = observations[0:5, :512]
Z

dask.array<getitem, shape=(5, 512, 128, 128, 3), dtype=float32, chunksize=(5, 512, 16, 16, 3), chunktype=numpy.ndarray>

In [6]:
start = time.time()
for i in Z:
    i.compute()

print(time.time() - start)

5.698678731918335


In [167]:
start = time.time()
Z.compute()

print(time.time() - start)

3.8444414138793945


In [8]:
def index_obs(l, r):
    return observations[l:r, :512].compute()

In [27]:
Q = []
client = concurrent.futures.ThreadPoolExecutor(16)

In [127]:
class PrefetchGenerator:
    def __init__(self, chunk_size=5, num_to_prefetch=32, ):
        self.Q = []
        self.client = concurrent.futures.ThreadPoolExecutor(16)
        
    def __del__(self):
        

    def __iter__(self):
        MAX_SIZE = 32
        I = 0

        while True:
            for _ in range(MAX_SIZE - len(self.Q)):
                l, r = I * chunk_size, I * chunk_size + chunk_size
                future = client.submit(index_obs, l, r)
                self.Q.append(future)
                I += 1

            for future in Q:
                if not future.running():
                    chunk = future.result()

                    for item in chunk:
                        yield item

                    Q.remove(future)
                    break

In [162]:
Q

[<Future at 0x7efb505b3cd0 state=finished returned ndarray>,
 <Future at 0x7efb50588750 state=finished returned ndarray>,
 <Future at 0x7efb505e2cd0 state=finished returned ndarray>,
 <Future at 0x7efb50328710 state=finished returned ndarray>,
 <Future at 0x7efb505e20d0 state=finished returned ndarray>,
 <Future at 0x7efb503be5d0 state=finished returned ndarray>,
 <Future at 0x7efb503bef50 state=finished returned ndarray>,
 <Future at 0x7efb505e2310 state=finished returned ndarray>,
 <Future at 0x7efb505e2090 state=finished returned ndarray>,
 <Future at 0x7efb505e2410 state=finished returned ndarray>,
 <Future at 0x7efb505cd790 state=finished returned ndarray>,
 <Future at 0x7efb505cdb90 state=finished returned ndarray>,
 <Future at 0x7efb505e2650 state=finished returned ndarray>,
 <Future at 0x7efb505e2710 state=finished returned ndarray>,
 <Future at 0x7efb505e27d0 state=finished returned ndarray>,
 <Future at 0x7efb53947850 state=finished returned ndarray>]

In [130]:
it = iter(generator())

In [165]:
%%time
val = next(it)
val.shape

CPU times: user 0 ns, sys: 2.11 ms, total: 2.11 ms
Wall time: 2.82 ms


(3, 512, 128, 128, 3)

In [ ]:
len(Q)

In [ ]:
client.shutdown()


 <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexingindexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing indexingindexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object gen

In [95]:
ds = tf.data.Dataset.from_generator(lambda: generator(8), tf.float32)

i = iter(ds)

In [98]:
%%time
t = next(i)
t.shape

CPU times: user 76.7 ms, sys: 161 ms, total: 238 ms
Wall time: 235 ms


TensorShape([8, 512, 128, 128, 3])

In [134]:
test

<generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
i

NameError: name 'test' is not defined

indexing  <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd

indexing<generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd51


indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing indexing 
indexing <generator object g

...

<generator object generator at 0x7efbd5151850><generator object generator at 0x7efbd5151850>indexing 

 <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexingindexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
i

indexingindexing<generator object generator at 0x7efbd5151850> 
<generator object generator at 0x7efbd5151850> indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing  <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5

<generator object generator at 0x7efbd5151850>indexingindexing<generator object generator at 0x7efbd5151850> indexingindexing   <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0

indexing <generator object generator at 0x7efbd5151850> indexingindexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing
<generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 

indexing <generator object generator at 0x7efbd5151850> 


indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850>
indexing <generator object generator at 0x7efbd5151850><generator object generator at 0x7efbd5151850><generator object generator at 0x7efbd5151850>indexing <generator object generator at 0x7efbd5151850>
i